# 지도 시각화

# 시군구별 인구 단계 구분도 만들기

# 단계 구분도란?
지역별 통계치를 색깔 차이로 표현한 지도

단계 구분도를 만들면 인구나 소득 같은 통계치가 지역별로 어떻게 다른지 쉽게 이해할 수 있다.

# 1. 데이터 준비하기

In [147]:
import json

geo = json.load(open("../data/SIG.geojson", encoding="utf-8"))

In [148]:
# 행정 구역 코드 출력
geo["features"][0]["properties"]

{'SIG_CD': '42110', 'SIG_ENG_NM': 'Chuncheon-si', 'SIG_KOR_NM': '춘천시'}

In [149]:
# 위도, 경도 좌표 출력
geo["features"][0]["geometry"]

{'type': 'MultiPolygon',
 'coordinates': [[[[127.58508551154958, 38.08062321552708],
    [127.58565575732702, 38.0802009066172],
    [127.58777905808203, 38.080354190085544],
    [127.58890487394689, 38.080881783588694],
    [127.59031267326897, 38.080596307998306],
    [127.59061778023133, 38.08053451807929],
    [127.59137292963024, 38.080476965118685],
    [127.59349548967889, 38.08031540227777],
    [127.594304166838, 38.080225164594665],
    [127.59529626584073, 38.08011430122607],
    [127.59683797157537, 38.079896764445195],
    [127.59703599139921, 38.07914773681181],
    [127.59756973764837, 38.07761547763786],
    [127.59794006383213, 38.07714541130494],
    [127.59816760846913, 38.07685681587356],
    [127.59847648881038, 38.076670284099194],
    [127.59956152109447, 38.076014995491754],
    [127.60022591131202, 38.07568645192285],
    [127.60034758173289, 38.075626138582486],
    [127.6014570759825, 38.07461119532391],
    [127.60157555080873, 38.07455485196505],
    [127.6

# 2. 시군구별 인구 데이터 준비하기

In [150]:
import pandas as pd

df_pop = pd.read_csv("../data/Population_SIG.csv")
df_pop.head()

,code,region,pop
0,11,서울특별시,9509458
1,11110,종로구,144683
2,11140,중구,122499
3,11170,용산구,222953
4,11200,성동구,285990


In [151]:
print(df_pop.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    278 non-null    int64 
 1   region  278 non-null    object
 2   pop     278 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 6.6+ KB
None


In [152]:
# 데이터 불러오기
df = pd.read_csv("../data/행정구역_시군구_별_주민등록세대수_20240728105926.csv")

# 컬럼명 변경 (M202406 2024.06 -> pop)
df = df.rename(columns={"M202406 2024.06": "pop"})

In [153]:
# 인구수가 0인 곳 제거
df_filtered = df[df["pop"] != 0]
df_filtered.head()

,A 행정구역(시군구)별,pop
0,00 전국,24057429
1,11 서울특별시,4486014
2,11110 종로구,72394
3,11140 중구,65273
4,11170 용산구,104187


In [154]:
# 지역코드와 지역 분리 (code / region)
df_filtered.loc[:, "code"] = df_filtered.loc[:, "A 행정구역(시군구)별"].str.split(
    " ", expand=True
)[0]
df_filtered.loc[:, "region"] = df_filtered.loc[:, "A 행정구역(시군구)별"].str.split(
    " ", expand=True
)[1]
df_filtered

/var/folders/dn/kpsvhr056zz6gvcnmgrkklzr0000gn/T/ipykernel_34343/3924675772.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:,'code'] = df_filtered.loc[:,'A 행정구역(시군구)별'].str.split(" ",expand=True)[0]
/var/folders/dn/kpsvhr056zz6gvcnmgrkklzr0000gn/T/ipykernel_34343/3924675772.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:,'region'] = df_filtered.loc[:,'A 행정구역(시군구)별'].str.split(" ",expand=True)[1]


,A 행정구역(시군구)별,pop,code,region
0,00 전국,24057429,00,전국
1,11 서울특별시,4486014,11,서울특별시
2,11110 종로구,72394,11110,종로구
3,11140 중구,65273,11140,중구
4,11170 용산구,104187,11170,용산구
...,...,...,...,...
288,48880 거창군,31219,48880,거창군
289,48890 합천군,23895,48890,합천군
290,50 제주특별자치도,314321,50,제주특별자치도
291,50110 제주시,225423,50110,제주시


In [155]:
# 컬럼 삭제
df_filtered.drop(columns=["A 행정구역(시군구)별"])

,pop,code,region
0,24057429,00,전국
1,4486014,11,서울특별시
2,72394,11110,종로구
3,65273,11140,중구
4,104187,11170,용산구
...,...,...,...
288,31219,48880,거창군
289,23895,48890,합천군
290,314321,50,제주특별자치도
291,225423,50110,제주시


In [156]:
# 컬럼 순서 변경
df_filtered = df_filtered[["code", "region", "pop"]]
df_filtered

,code,region,pop
0,00,전국,24057429
1,11,서울특별시,4486014
2,11110,종로구,72394
3,11140,중구,65273
4,11170,용산구,104187
...,...,...,...
288,48880,거창군,31219
289,48890,합천군,23895
290,50,제주특별자치도,314321
291,50110,제주시,225423


In [157]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 282 entries, 0 to 292
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    282 non-null    object
 1   region  282 non-null    object
 2   pop     282 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 8.8+ KB


# 3. 단계 구분도 만들기
## folium 설치

In [159]:
import folium

folium.Map(location=[35.97, 127.7], zoom_start=8)  # 지도 중심 좌표  # 확대 단계

In [162]:
map_sig = folium.Map(
    location=[35.95, 127.1],  # 지도 중심 좌표
    zoom_start=8,  # 확대 단계
    tiles="cartodbpositron",  # 지도 종류
)
map_sig

# 구분도 만들기

In [165]:
# folium.Choropleth()
# geo_data   : 지도 데이터
# data       : 색깔로 표현할 통계 데이터
# columns    : 통계 데이터의 행정 구역 코드 변수, 색깔로 표현할 변수
# key_on     : 지도 데이터의 행정 구역 코드

folium.Choropleth(
    geo_data=geo,
    data=df_pop,
    columns=("code", "pop"),
    key_on="feature.properties.SIG_CD",
).add_to(map_sig)

map_sig